In [21]:

class Scalar1:

    def __init__(self, v):
        self.v = v
        self.grad = 0        

    def __repr__(self):
        return f"Scalar(v={self.v}, grad={self.grad})"

    def __add__(self, o):
        self.grad += 1
        o.grad += 1
        return Scalar(self.v + o.v)

    def __mul__(self, o):
        self.grad += o.v
        o.grad += self.v
        return Scalar(self.v * o.v)

a = Scalar1(2)
b = Scalar1(3)
c = Scalar1(4)
z = a + b * c
z

Scalar(v=14, grad=0)

In [22]:

class Scalar:

    def __init__(self, v, children=tuple(), back=lambda _: None, op=""):
        self.v = v
        self.children = children
        self.back = back
        self.grad = 0
        self.op = op

    def __repr__(self):
        return f"Scalar(v={self.v}, grad={self.grad})"

    def __add__(self, o):
        def __back():
            self.grad += 1
            o.grad += 1
        return Scalar(self.v + o.v, children=(self, o), back=__back, op="+")

    def __mul__(self, o):
        def __back():
            self.grad += o
            o.grad += self.v
        return Scalar(self.v * o.v, children=(self, o), back=__back, op="*")

    def back(self):
        pass

a = Scalar(2)
b = Scalar(3)
c = Scalar(4)
z = a + b * c
c

Scalar(v=4, grad=0)